In [88]:
import pandas as pd
import numpy as np
from datetime import datetime

In [89]:
# Imports data from csv file
df= pd.read_csv('/Users/nieksonneveld/code/nieksonneveld/election-predictor/raw_data/All Constituency General Election Results.csv')

In [90]:
df.head()

,Election Year,Boundary Year,Constituency,Candidate,Party,Votes,Vote Share
0,1955,1955,Ashfield,Alan S Plane,Conservative,12836,28.06
1,1955,1955,Ashfield,William Warbey,Labour,32905,71.94
2,1955,1955,Bassetlaw,Kathleen Voilet Maiden,Conservative,19375,41.89
3,1955,1955,Bassetlaw,Frederick Bellenger,Labour,26873,58.11
4,1955,1955,Belper,John Twells,Conservative,24115,44.40


In [91]:
df_2004 = df[df['Election Year'] > 2003].copy()

In [102]:
df_2004['Constituency'].value_counts()

Constituency
Hackney South and Shoreditch        47
Bethnal Green and Bow               41
Camberwell and Peckham              41
Sheffield Central                   40
Cities of London and Westminster    40
                                    ..
Ealing Acton and Shepherd’s Bush     4
Knowsley South                       3
Vale of York                         3
Barnsley West and Penistone          3
Selby                                3
Name: count, Length: 731, dtype: int64

In [98]:
gb_parties_abbr = ['BRX', 'CON', 'GRE', 'LAB', 'LIB', 'OTH', 'PLC', 'SNP', 'UKI']
gb_parties = ['The Brexit Party','Conservative','Green Party','Labour','Liberal Democrats','Other','Plaid Cymru','SNP','UK Independence Party (UKIP)']

In [100]:
df_2004['Party'].replace(gb_parties,gb_parties_abbr)

30087    British National Party
30088                       CON
30089                       LIB
30090                       UKI
30091                   Veritas
                  ...          
48366                       LIB
48367                       CON
48368                       BRX
48369                       LAB
48370                       PLC
Name: Party, Length: 18284, dtype: object

In [65]:
df_2004['new_index'] = df_2004_on['Election Year'].astype(str) + '_' + df_2004_on['Constituency']

In [66]:
df_2004_indexed = df_2004_on.set_index('new_index').rename_axis(None)

In [ ]:
# Get Chris' naming convention for parties and map mine

In [67]:
df_2004_indexed

,Election Year,Boundary Year,Constituency,Candidate,Party,Votes,Vote Share
2005_Amber Valley,2005,2005,Amber Valley,Paul Snell,British National Party,1243,2.6
2005_Amber Valley,2005,2005,Amber Valley,Gillian Shaw,Conservative,16318,34.4
2005_Amber Valley,2005,2005,Amber Valley,Kate Smith,Liberal Democrats,6225,13.1
2005_Amber Valley,2005,2005,Amber Valley,Hugh Price,UK Independence Party (UKIP),788,1.7
2005_Amber Valley,2005,2005,Amber Valley,Alex Stevenson,Veritas,1224,2.6
...,...,...,...,...,...,...,...
2019_Wrexham,2019,2010,Wrexham,Tim Sly,Liberal Democrats,1447,4.3
2019_Ynys Mon,2019,2010,Ynys Mon,Virginia Crosbie,Conservative,12959,35.5
2019_Ynys Mon,2019,2010,Ynys Mon,Helen Jenner,The Brexit Party,2184,6.0
2019_Ynys Mon,2019,2010,Ynys Mon,Mary Roberts,Labour,10991,30.1


In [85]:
pivoted = df_2004_indexed.pivot_table(index=df_2004_indexed.index, columns=['Party'], values=['Votes']).droplevel(0, axis=1)
pivoted

Party,A Vote Against MP Expense Abuse,Above and Beyond,Advance,Al-Zebabist Nation of Ooog,All People’s Party,All The South Party,Alliance,Alliance For Change,Alliance for Change,Alliance for Green Socialism,...,World,World Peace Through Song,Wycombe Independents,Xtraordinary People,Yeshua,Yorkshire First,You Party,Young People’s Party YPP,Your Party,Your Right to Democracy
2005_Aberavon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005_Aberdeen North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005_Aberdeen South,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005_Aberdeenshire West and Kincardine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005_Airdrie and Shotts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019_Yeovil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019_Ynys Mon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019_York Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019_York Outer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
pivoted[['Conservative', 'Labour']]

Party,Conservative,Labour
2005_Aberavon,3064.0,18077.0
2005_Aberdeen North,3456.0,15557.0
2005_Aberdeen South,7134.0,15272.0
2005_Aberdeenshire West and Kincardine,11814.0,5470.0
2005_Airdrie and Shotts,3271.0,19568.0
...,...,...
2019_Yeovil,34588.0,3761.0
2019_Ynys Mon,12959.0,10991.0
2019_York Central,13767.0,27312.0
2019_York Outer,27324.0,17339.0
